In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

import pandas as pd
import re

def extract_centroid_um_data(file_path):
    """
    Extracts time-of-day and centroid positions in µm from a Thorlabs beam profile file.

    Parameters:
        file_path (str): Path to the input text file.

    Returns:
        pd.DataFrame: DataFrame with columns ['Time', 'Centroid Position X [µm]', 'Centroid Position Y [µm]']
    """
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()

    # Locate the header line
    start_idx = next(i for i, line in enumerate(lines) if line.strip().startswith("Measurement"))

    data = []
    for line in lines[start_idx + 1:]:
        parts = re.split(r'\t+', line.strip())
        if len(parts) < 10:
            continue
        full_time = parts[1]  # e.g., "7/1/2025 15:07:03.988"
        try:
            time_only = full_time.split()[-1]  # Keep only "15:07:03.988"
            centroid_x_um = float(parts[8])
            centroid_y_um = float(parts[9])
            data.append([time_only, centroid_x_um, centroid_y_um])
        except (ValueError, IndexError):
            continue

    df = pd.DataFrame(data, columns=["Time", "Centroid Position X [µm]", "Centroid Position Y [µm]"])
    return df

In [9]:
df1 = extract_centroid_um_data(r"C:\ACME_analysis\jittering STIRAP\Vertical STIRAP 2min high_#001.txt")
df2 = extract_centroid_um_data(r"C:\ACME_analysis\jittering STIRAP\Vertical STIRAP 2min low_#001.txt")

In [11]:
px.line(df1, x="Time", y=["Centroid Position X [µm]"], title="Centroid Positions Over Time")

In [13]:
px.line(df2, x="Time", y=["Centroid Position X [µm]"], title="Centroid Positions Over Time")

In [20]:
df1_ma = df1['Centroid Position X [µm]'].rolling(window=1000).mean()
df2_ma = df2['Centroid Position X [µm]'].rolling(window=1000).mean()
# two centroid x positions on same plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['Time'], y=df1_ma, mode='lines',
                        name='High Power', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df2['Time'], y=df2_ma, mode='lines', name='Low Power', line=dict(color='red')))
fig.update_layout(title='Centroid Positions Over Time (Moving Average)', xaxis_title='Time', yaxis_title='Centroid Position X [µm]')
fig.show()
